<a href="https://colab.research.google.com/github/JeanPinhata/PROGRAMA-O-DE-LINGUAGEM-NATURAL-/blob/main/EXTRA%C3%87%C3%83O_DE_ENTIDADES_NOMEADAS_COM_LLM_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets seqeval scikit-learn
!pip install --upgrade transformers

import requests
import re
from io import StringIO
import pandas as pd
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoTokenizer, pipeline
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt

In [ ]:
# Baixar dados
url = "https://raw.githubusercontent.com/davidsbatista/NER-datasets/refs/heads/master/Portuguese/leNER-Br/dev.conll"
response = requests.get(url)
raw_text = response.text

# Parse
sentences = []
ner_tags = []
tokens = []
tags = []

for line in raw_text.splitlines():
    line = line.strip()
    if line == "":
        if tokens:
            sentences.append(tokens)
            ner_tags.append(tags)
            tokens, tags = [], []
    else:
        parts = re.split(r"\s+", line)
        if len(parts) == 2:
            token, tag = parts
            tokens.append(token)
            tags.append(tag)

if tokens:
    sentences.append(tokens)
    ner_tags.append(tags)

examples = [{"tokens": s, "ner_tags": t} for s, t in zip(sentences, ner_tags)]

print(f"Total de sentenças: {len(examples)}")
print("Exemplo:", examples[0])


In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

train_data, test_data = train_test_split(examples, test_size=0.2, random_state=42)

dataset = DatasetDict({
    "train": Dataset.from_list(train_data),
    "test": Dataset.from_list(test_data),
})


In [ ]:
model_checkpoint = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

# Mapear rótulos
label_list = sorted(list({tag for ex in examples for tag in ex["ner_tags"]}))
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

# Função de tokenização
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(
        example["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",  # <- importante!
        max_length=128,        # <- para uniformizar o tamanho
    )

    word_ids = tokenized_inputs.word_ids()
    label_ids = []
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:
            label_ids.append(label2id[example["ner_tags"][word_idx]])
        else:
            label_ids.append(-100)  # sub-palavras são ignoradas
        previous_word_idx = word_idx

    tokenized_inputs["labels"] = label_ids
    return tokenized_inputs

# Tokenizar dataset
tokenized_dataset = dataset.map(tokenize_and_align_labels)
print("Entidades reconhecidas:", label_list)

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    "neuralmind/bert-base-portuguese-cased",
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir="./ner-bert-learnbr",
    eval_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
)

# Métricas
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_preds = [
        [id2label[p] for (p, l) in zip(pred, lab) if l != -100]
        for pred, lab in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(pred, lab) if l != -100]
        for pred, lab in zip(predictions, labels)
    ]
    return {
        "precision": precision_score(true_labels, true_preds),
        "recall": recall_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds),
        "accuracy": accuracy_score(true_labels, true_preds),
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
epochs = list(range(1, 11))
f1_scores = [
    0.547731,
    0.788991,
    0.849558,
    0.876336,
    0.891975,
    0.888550,
    0.892638,
    0.888208,
    0.902628,
    0.895385
]

plt.figure(figsize=(10, 5))
plt.plot(epochs, f1_scores, marker='o', linestyle='-', color='skyblue')
plt.title("Evolução do F1-score por Época")
plt.xlabel("Época")
plt.ylabel("F1-score")
plt.xticks(epochs)
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

epochs = list(range(1, 11))

precision = [
    0.501377,
    0.789318,
    0.858859,
    0.863636,
    0.872340,
    0.884848,
    0.886850,
    0.875000,
    0.902141,
    0.914373
]

recall = [
    0.568750,
    0.831250,
    0.893750,
    0.890625,
    0.896875,
    0.912500,
    0.906250,
    0.896875,
    0.921875,
    0.934375
]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
ax1.plot(epochs, precision, marker='o', linestyle='-', color='blue')
ax1.set_title("Evolução da Precisão por Época")
ax1.set_xlabel("Época")
ax1.set_ylabel("Precision Score")
ax1.set_ylim(0.4, 1.0)
ax1.set_xticks(epochs)
ax1.grid(True, linestyle='--', alpha=0.7)

ax2 = axes[1]
ax2.plot(epochs, recall, marker='x', linestyle='-', color='red')
ax2.set_title("Evolução do Recall por Época")
ax2.set_xlabel("Época")
ax2.set_ylabel("Recall Score")
ax2.set_ylim(0.4, 1.0)
ax2.set_xticks(epochs)
ax2.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


In [ ]:
trainer.save_model("bert-lenerbr-final-f1-0_92")


In [ ]:
model_path = "/content/bert-lenerbr-final-f1-0_92"

if not os.path.exists(model_path):
    print(f"Erro: O diretório do modelo '{model_path}' não foi encontrado.")
    print("Por favor, verifique o caminho e certifique-se de que o modelo foi salvo corretamente.")
else:
    print(f"Carregando modelo do caminho: {model_path}")
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        ner_pipeline = pipeline(
            "ner",
            model=model_path,
            tokenizer=tokenizer,
            aggregation_strategy="simple"
        )
        print("Modelo e pipeline carregados com sucesso!")

        text = "Em 18 de julho de 2023, a advogada Ana Paula Resende da Ordem dos Advogados do Brasil, localizada em Curitiba, argumentou com base no Artigo 14 da Constituição Federal e citou o Recurso Especial 123.456/SP, reforçando seu ponto."

        try:
            entities = ner_pipeline(text)

            if entities:
                print("\nEntidades Extraídas:")
                for entity in entities:
                    print(f"'{entity['word']}' -> Tipo: {entity['entity_group']}")
            else:
                print("Nenhuma entidade foi encontrada nesta frase.")
        except Exception as e:
            print(f"Ocorreu um erro ao processar a frase: {e}")
            print("Por favor, tente novamente ou verifique se o modelo foi carregado corretamente.")

    except Exception as e:
        print(f"Erro ao carregar o modelo ou pipeline: {e}")
        print("Verifique se as bibliotecas estão instaladas e se o modelo no caminho especificado é válido.")




In [ ]:
from transformers import pipeline, AutoTokenizer
import os

# --- 1. Caminho para o modelo salvo (ajuste conforme seu diretório) ---
model_path = "/content/bert-lenerbr-final-f1-0_92"

if not os.path.exists(model_path):
    print(f"Erro: O diretório do modelo '{model_path}' não foi encontrado.")
    print("Por favor, verifique o caminho e certifique-se de que o modelo foi salvo corretamente.")
else:
    print(f"Carregando modelo do caminho: {model_path}")
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        ner_pipeline = pipeline(
            "ner",
            model=model_path,
            tokenizer=tokenizer,
            aggregation_strategy="simple"
        )
        print("Modelo e pipeline carregados com sucesso!")

        # --- 3. Loop para entrada do usuário ---
        print("\n--- Modo Interativo NER ---")
        print("Digite uma frase para extrair entidades. Digite 'sair' para encerrar.")

        while True:
            text = input("\nSua frase: ")
            if text.lower() == 'sair':
                print("Encerrando o programa.")
                break
            if not text.strip():
                print("Por favor, digite uma frase válida.")
                continue

            # --- 4. Executar o modelo ---
            try:
                entities = ner_pipeline(text)

                # --- 5. Exibir as entidades extraídas (sem o score) ---
                if entities:
                    print("\nEntidades Extraídas:")
                    for entity in entities:
                        print(f"'{entity['word']}' -> Tipo: {entity['entity_group']}")
                else:
                    print("Nenhuma entidade foi encontrada nesta frase.")
            except Exception as e:
                print(f"Ocorreu um erro ao processar a frase: {e}")
                print("Por favor, tente novamente ou verifique se o modelo foi carregado corretamente.")

    except Exception as e:
        print(f"Erro ao carregar o modelo ou pipeline: {e}")
        print("Verifique se as bibliotecas estão instaladas e se o modelo no caminho especificado é válido.")

  #Em 18 de julho de 2023, a advogada Ana Paula Resende da Ordem dos Advogados do Brasil, localizada em Curitiba, argumentou com base no Artigo 14 da Constituição Federal e citou o Recurso Especial 123.456/SP, reforçando seu ponto.#